# 예제 - QLoRA (Quantized LoRA)

## 1. 환경 설정

In [1]:
# 1. 환경 설정: 필요한 라이브러리를 설치하고 환경을 설정합니다.
!pip install -q transformers==4.56.1 peft==0.17.1 accelerate==1.10.1 bitsandbytes==0.47.0 trl==0.22.2


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


## 2. 모델 및 토크나이저 로드

In [2]:
# 2. 모델 및 토크나이저 로드: "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct" 모델과 해당 토크나이저를 로드합니다.
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_id = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"":0}, # Ensure the model is loaded on the GPU
    trust_remote_code=True
)
model.config.use_cache = False
model.config.pretraining_tp = 1 # Not needed for this model, but good practice

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token # Or use a specific pad token if available in the tokenizer config

config.json: 0.00B [00:00, ?B/s]

configuration_exaone.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct:
- configuration_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_exaone.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct:
- modeling_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

## 3. 데이터셋 로드 및 전처리

In [3]:
# 3. 데이터셋 로드 및 전처리: NSMC 데이터셋을 로드하고 모델 학습에 적합한 형태로 전처리합니다.
from datasets import load_dataset, Dataset
import pandas as pd

# NSMC 데이터셋 파일 URL
train_data_url = "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt"
test_data_url = "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt"

try:
    # 데이터 다운로드 및 pandas DataFrame으로 로드
    train_df = pd.read_csv(train_data_url, sep='\t').dropna()
    test_df = pd.read_csv(test_data_url, sep='\t').dropna()

    # pandas DataFrame을 Hugging Face Dataset으로 변환
    train_dataset = Dataset.from_pandas(train_df)
    test_dataset = Dataset.from_pandas(test_df)

    # 학습에 사용할 데이터셋 선택 (여기서는 학습 데이터셋의 일부를 사용)
    # 메모리 제약을 고려하여 작은 크기의 데이터셋 사용
    dataset = train_dataset.select(range(1000)) # 학습 데이터셋의 처음 1000개 예제 사용

    print("NSMC dataset loaded.")
    print(dataset)

    # Function to format the dataset for the model
    # NSMC 데이터셋은 'document'와 'label' 컬럼을 가집니다.
    def format_dataset(example):
        # 'document'와 'label'을 사용하여 모델 입력 형식에 맞게 포맷팅
        # 예: 긍정/부정 리뷰 분류 태스크에 맞게 형식을 조정합니다.
        # 여기서는 간단하게 'document'와 'label'을 합쳐 텍스트 생성 형태로 만듭니다.
        # 실제 fine-tuning 태스크에 따라 이 부분은 수정이 필요합니다.
        label_text = "긍정" if example['label'] == 1 else "부정"
        return {"text": f"리뷰: {example['document']}\n판별: {label_text}"}


    # Apply formatting and tokenize
    dataset = dataset.map(format_dataset)

    def tokenize_function(examples):
        # Ensure padding and truncation
        return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512) # Adjust max_length as needed

    tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)
    print("Dataset loaded and preprocessed.")
    print(tokenized_dataset)

except Exception as e:
    print(f"Error loading or processing NSMC dataset: {e}")
    dataset = None
    print("Could not load NSMC dataset. Please check the URLs and your internet connection.")


if not dataset:
     print("No dataset loaded. Please ensure the dataset is loaded correctly to proceed with training.")

NSMC dataset loaded.
Dataset({
    features: ['id', 'document', 'label', '__index_level_0__'],
    num_rows: 1000
})


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset loaded and preprocessed.
Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1000
})


## 4. Qlora 설정

In [4]:
# 4. Qlora 설정: peft 라이브러리를 사용하여 QLoRA 설정을 정의합니다.
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

if dataset: # Only proceed if a dataset was successfully loaded
    # Prepare model for k-bit training
    model = prepare_model_for_kbit_training(model)

    # Define LoRA configuration
    lora_config = LoraConfig(
        r=16, # LoRA attention dimension
        lora_alpha=16, # Alpha parameter for LoRA scaling
        lora_dropout=0.05, # Dropout probability for LoRA layers
        bias="none", # Bias type: 'none', 'all', or 'lora_only'
        task_type="CAUSAL_LM", # Task type
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], # Modules to apply LoRA to
    )

    # Get PEFT model
    model = get_peft_model(model, lora_config)

    print("QLoRA configuration applied.")
    model.print_trainable_parameters()
else:
    print("Skipping QLoRA configuration as no dataset was loaded.")

QLoRA configuration applied.
trainable params: 5,529,600 || all params: 2,410,856,960 || trainable%: 0.2294


## 5. 모델 학습

In [5]:
# 5. 모델 학습: 설정된 QLoRA와 데이터셋을 사용하여 모델을 학습합니다.
from transformers import TrainingArguments

if dataset: # Only proceed if a dataset was successfully loaded
    training_arguments = TrainingArguments(
        output_dir="./results", # Output directory
        num_train_epochs=1, # Number of training epochs
        per_device_train_batch_size=4, # Batch size per device during training
        gradient_accumulation_steps=1, # Number of updates steps to accumulate the gradients for
        optim="paged_adamw_32bit", # Optimizer to use
        logging_steps=10, # Log every x updates steps
        learning_rate=2e-4, # Learning rate
        weight_decay=0.001, # Weight decay
        fp16=False, # Use mixed precision
        bf16=True, # Use bfloat16 precision
        max_grad_norm=0.3, # Maximum gradient normal
        warmup_ratio=0.03, # Ratio of warmup steps for learning rate scheduler
        lr_scheduler_type="constant", # Learning rate scheduler
        report_to="none", # Reporting to experiment tracking platforms
        save_strategy="epoch", # Save the model checkpoint every epoch
    )

    from trl import SFTTrainer

    trainer = SFTTrainer(
        model=model,
        train_dataset=tokenized_dataset,
        peft_config=lora_config,
        # dataset_text_field="text", # Removed as it's not a recognized argument
        # max_seq_length=512, # Maximum sequence length - Removed as it's not a recognized argument
        # tokenizer=tokenizer, # Removed as it's not a recognized argument
        args=training_arguments,
        # packing=False, # Pack multiple examples into one sequence - Removed as it's not a recognized argument
    )

    # Start training
    print("Starting model training...")
    trainer.train()
    print("Training finished.")
else:
     print("Skipping model training as no dataset was loaded.")

The repository LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct .
 You can inspect the repository content at https://hf.co/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Truncating train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Starting model training...


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,9.915900
20,6.767100
30,1.655700
40,0.224200
50,0.219100
60,0.154400
70,0.202200
80,0.165200
90,0.233900
100,0.168700


Training finished.


## 6. 모델 저장


In [6]:
# PEFT 모델 저장
if dataset: # 학습이 완료된 경우에만 저장
    # 1. PEFT 어댑터만 저장 (가벼운 방식)
    model.save_pretrained("./peft_model")
    tokenizer.save_pretrained("./peft_model")
    print("PEFT 어댑터가 './peft_model' 디렉토리에 저장되었습니다.")
    
    # 2. 전체 모델 저장 (더 큰 용량이지만 완전한 모델)
    # model.save_pretrained("./full_model")
    # tokenizer.save_pretrained("./full_model")
    # print("전체 모델이 './full_model' 디렉토리에 저장되었습니다.")
    
    # 3. Hugging Face Hub에 업로드 (선택사항)
    # model.push_to_hub("your-username/exaone-peft-model")
    # tokenizer.push_to_hub("your-username/exaone-peft-model")
    # print("모델이 Hugging Face Hub에 업로드되었습니다.")
else:
    print("학습된 모델이 없어 저장할 수 없습니다.")


PEFT 어댑터가 './peft_model' 디렉토리에 저장되었습니다.


## 7. 저장된 모델 로드


In [7]:
# 저장된 PEFT 모델 로드
import os
from peft import PeftModel

# 저장된 모델이 있는지 확인
if os.path.exists("./peft_model"):
    print("저장된 PEFT 모델을 로드합니다...")
    
    # 1. 기본 모델 로드 (원본 모델)
    base_model_id = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"
    
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_id,
        quantization_config=bnb_config,
        device_map={"":0},
        trust_remote_code=True
    )
    
    # 2. 토크나이저 로드
    loaded_tokenizer = AutoTokenizer.from_pretrained("./peft_model", trust_remote_code=True)
    loaded_tokenizer.pad_token = loaded_tokenizer.eos_token
    
    # 3. PEFT 모델 로드
    loaded_model = PeftModel.from_pretrained(base_model, "./peft_model")
    
    print("PEFT 모델이 성공적으로 로드되었습니다!")
    print(f"모델 정보: {loaded_model}")
    
else:
    print("저장된 PEFT 모델을 찾을 수 없습니다. 먼저 모델을 학습하고 저장해주세요.")


저장된 PEFT 모델을 로드합니다...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PEFT 모델이 성공적으로 로드되었습니다!
모델 정보: PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): ExaoneForCausalLM(
      (transformer): ExaoneModel(
        (wte): Embedding(102400, 2560, padding_idx=0)
        (drop): Dropout(p=0.0, inplace=False)
        (h): ModuleList(
          (0-29): 30 x ExaoneBlock(
            (ln_1): ExaoneRMSNorm()
            (attn): ExaoneAttention(
              (attention): ExaoneSdpaAttention(
                (rotary): ExaoneRotaryEmbedding()
                (k_proj): lora.Linear4bit(
                  (base_layer): Linear4bit(in_features=2560, out_features=640, bias=False)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=2560, out_features=16, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=16, out_features=640, bia

## 8. 추론


### 함수 정의

In [8]:
# 텍스트 생성을 위한 함수
def generate_text(model, tokenizer, prompt, max_length=100, temperature=0.7, top_p=0.9, num_return_sequences=1):
    # 입력 텍스트 토크나이징
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # 텍스트 생성
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1,
            num_return_sequences=num_return_sequences
        )
    
    # 생성된 텍스트들 디코딩
    generated_texts = []
    for output in outputs:
        generated_text = tokenizer.decode(output, skip_special_tokens=True)
        # 입력 프롬프트 부분 제거하고 생성된 부분만 반환
        generated_text = generated_text[len(prompt):].strip()
        generated_texts.append(generated_text)
    
    return generated_texts

In [9]:
# 리뷰의 감정을 분석하는 함수
def evaluate_sentiment(review, model, tokenizer):
    prompt = f"리뷰: {review}\n판별:"
    results = generate_text(model, tokenizer, prompt, max_length=20, temperature=0.3)
    
    # 결과에서 감정 분석
    result_text = results[0].lower()
    if "긍정" in result_text or "좋" in result_text or "만족" in result_text:
        return "긍정"
    elif "부정" in result_text or "나쁘" in result_text or "별로" in result_text:
        return "부정"
    else:
        return "중립"

### 추론 예제 실행

In [10]:
if 'loaded_model' in locals() and 'loaded_tokenizer' in locals():
    print("=== PEFT 모델 추론 예제 ===\n")
    
    # 테스트할 리뷰 텍스트들
    test_reviews = [
        "이 영화 정말 재미있었어요!",
        "음식이 맛없고 서비스도 별로였습니다.",
        "제품 품질이 좋고 배송도 빨랐어요.",
        "가격이 너무 비싸고 만족스럽지 않습니다.",
        "배우들의 연기가 훌륭했습니다.",
        "스토리가 너무 지루하고 예측 가능했습니다."
    ]
    
    print("리뷰 감정 분석 결과:")
    print("=" * 60)
    
    correct_predictions = 0
    total_predictions = len(test_reviews)
    
    for i, review in enumerate(test_reviews, 1):
        print(f"\n테스트 {i}: {review}")
        
        # 감정 분석 실행
        predicted_sentiment = evaluate_sentiment(review, loaded_model, loaded_tokenizer)
        
        # 실제 감정 (간단한 규칙 기반)
        if any(word in review for word in ["재미있", "좋", "훌륭", "만족"]):
            actual_sentiment = "긍정"
        elif any(word in review for word in ["맛없", "별로", "지루", "비싸"]):
            actual_sentiment = "부정"
        else:
            actual_sentiment = "중립"
        
        # 정확도 계산
        is_correct = predicted_sentiment == actual_sentiment
        if is_correct:
            correct_predictions += 1
        
        print(f"   예측: {predicted_sentiment} | 실제: {actual_sentiment} | {'✅' if is_correct else '❌'}")
    
    # 전체 정확도 출력
    accuracy = correct_predictions / total_predictions * 100
    print(f"\n전체 정확도: {accuracy:.1f}% ({correct_predictions}/{total_predictions})")
    
else:
    print("로드된 모델이 없습니다. 먼저 모델을 로드해주세요.")


=== PEFT 모델 추론 예제 ===

리뷰 감정 분석 결과:

테스트 1: 이 영화 정말 재미있었어요!
   예측: 긍정 | 실제: 긍정 | ✅

테스트 2: 음식이 맛없고 서비스도 별로였습니다.
   예측: 부정 | 실제: 부정 | ✅

테스트 3: 제품 품질이 좋고 배송도 빨랐어요.
   예측: 긍정 | 실제: 긍정 | ✅

테스트 4: 가격이 너무 비싸고 만족스럽지 않습니다.
   예측: 부정 | 실제: 긍정 | ❌

테스트 5: 배우들의 연기가 훌륭했습니다.
   예측: 긍정 | 실제: 긍정 | ✅

테스트 6: 스토리가 너무 지루하고 예측 가능했습니다.
   예측: 부정 | 실제: 부정 | ✅

전체 정확도: 83.3% (5/6)


## 9. 배치 추론
- 여러 텍스트를 한번에 처리하는 효율적인 방법

### 함수 정의

In [11]:
import time
from typing import List

def batch_inference(model, tokenizer, texts: List[str], batch_size: int = 4):
    results = []
    
    print(f"배치 크기 {batch_size}로 {len(texts)}개 텍스트 처리 중...")
    start_time = time.time()
    
    # 텍스트를 배치 단위로 나누어 처리
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        batch_prompts = [f"리뷰: {text}\n판별:" for text in batch_texts]
        
        # 배치 토크나이징
        batch_inputs = tokenizer(
            batch_prompts, 
            return_tensors="pt", 
            padding=True, 
            truncation=True, 
            max_length=512
        ).to(model.device)
        
        # 배치 추론
        with torch.no_grad():
            batch_outputs = model.generate(
                **batch_inputs,
                max_length=batch_inputs['input_ids'].shape[1] + 20,  # 입력 길이 + 20토큰
                temperature=0.3,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                repetition_penalty=1.1
            )
        
        # 배치 결과 처리
        for j, output in enumerate(batch_outputs):
            generated_text = tokenizer.decode(output, skip_special_tokens=True)
            # 입력 프롬프트 부분 제거
            prompt_length = len(batch_prompts[j])
            generated_text = generated_text[prompt_length:].strip()
            
            # 감정 분석
            sentiment = "긍정" if any(word in generated_text.lower() for word in ["긍정", "좋", "만족"]) else "부정"
            results.append({
                'text': batch_texts[j],
                'generated': generated_text,
                'sentiment': sentiment
            })
        
        print(f"   처리 완료: {min(i + batch_size, len(texts))}/{len(texts)}")
    
    end_time = time.time()
    processing_time = end_time - start_time
    
    print(f"총 처리 시간: {processing_time:.2f}초")
    print(f"평균 처리 속도: {len(texts)/processing_time:.2f} 텍스트/초")
    
    return results

### 배치 추론 실행

In [12]:
if 'loaded_model' in locals() and 'loaded_tokenizer' in locals():
    print("=== 배치 추론 예제 ===\n")
    
    # 더 많은 테스트 데이터
    large_test_reviews = [
        "이 영화는 정말 재미있고 감동적이었습니다.",
        "음식이 맛없고 서비스도 별로였습니다.",
        "제품 품질이 좋고 배송도 빨랐어요.",
        "가격이 너무 비싸고 만족스럽지 않습니다.",
        "배우들의 연기가 훌륭했습니다.",
        "스토리가 너무 지루하고 예측 가능했습니다.",
        "음식점 분위기가 좋고 직원들이 친절했습니다.",
        "배송이 늦고 포장도 엉망이었습니다.",
        "상품이 예상보다 훨씬 좋았습니다.",
        "고객 서비스가 형편없었습니다.",
        "가격 대비 품질이 매우 만족스럽습니다.",
        "환불 처리도 빠르고 친절했습니다.",
        "제품 설명과 실제가 다릅니다.",
        "추천하고 싶은 좋은 상품입니다.",
        "배송비가 너무 비싸네요."
    ]
    
    # 배치 크기별 성능 비교
    batch_sizes = [1, 4, 8]
    
    for batch_size in batch_sizes:
        print(f"\n🔧 배치 크기: {batch_size}")
        print("-" * 40)
        
        results = batch_inference(loaded_model, loaded_tokenizer, large_test_reviews, batch_size)
        
        # 결과 요약
        positive_count = sum(1 for r in results if r['sentiment'] == '긍정')
        negative_count = sum(1 for r in results if r['sentiment'] == '부정')
        
        print(f"결과 요약:")
        print(f"   긍정: {positive_count}개")
        print(f"   부정: {negative_count}개")
        print()
        
        # 처음 3개 결과만 상세 출력
        print("상세 결과 (처음 3개):")
        for i, result in enumerate(results[:3]):
            print(f"   {i+1}. {result['text']}")
            print(f"      → {result['sentiment']}: {result['generated']}")
            print()
        
        print("=" * 60)
        
else:
    print("로드된 모델이 없습니다. 먼저 모델을 로드해주세요.")


=== 배치 추론 예제 ===


🔧 배치 크기: 1
----------------------------------------
배치 크기 1로 15개 텍스트 처리 중...
   처리 완료: 1/15
   처리 완료: 2/15
   처리 완료: 3/15
   처리 완료: 4/15
   처리 완료: 5/15
   처리 완료: 6/15
   처리 완료: 7/15
   처리 완료: 8/15
   처리 완료: 9/15
   처리 완료: 10/15
   처리 완료: 11/15
   처리 완료: 12/15
   처리 완료: 13/15
   처리 완료: 14/15


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


   처리 완료: 15/15
총 처리 시간: 2.33초
평균 처리 속도: 6.43 텍스트/초
결과 요약:
   긍정: 8개
   부정: 7개

상세 결과 (처음 3개):
   1. 이 영화는 정말 재미있고 감동적이었습니다.
      → 긍정: 긍정

   2. 음식이 맛없고 서비스도 별로였습니다.
      → 부정: 부정

   3. 제품 품질이 좋고 배송도 빨랐어요.
      → 긍정: 긍정


🔧 배치 크기: 4
----------------------------------------
배치 크기 4로 15개 텍스트 처리 중...


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


   처리 완료: 4/15
   처리 완료: 8/15


...

   처리 완료: 12/15


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


   처리 완료: 15/15
총 처리 시간: 1.33초
평균 처리 속도: 11.30 텍스트/초
결과 요약:
   긍정: 7개
   부정: 8개

상세 결과 (처음 3개):
   1. 이 영화는 정말 재미있고 감동적이었습니다.
      → 긍정: 긍정

   2. 음식이 맛없고 서비스도 별로였습니다.
      → 부정: In

   3. 제품 품질이 좋고 배송도 빨랐어요.
      → 긍정: #긍정


🔧 배치 크기: 8
----------------------------------------
배치 크기 8로 15개 텍스트 처리 중...


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


   처리 완료: 8/15
   처리 완료: 15/15
총 처리 시간: 0.78초
평균 처리 속도: 19.18 텍스트/초
결과 요약:
   긍정: 5개
   부정: 10개

상세 결과 (처음 3개):
   1. 이 영화는 정말 재미있고 감동적이었습니다.
      → 긍정: 긍정

   2. 음식이 맛없고 서비스도 별로였습니다.
      → 부정: Thematic: 부정

   3. 제품 품질이 좋고 배송도 빨랐어요.
      → 부정: I



## 10. 모델 성능 비교
- 원본 모델 vs PEFT 모델

### 함수 정의

In [13]:
import torch
import time
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

def load_original_model():
    print("원본 모델 로드 중...")
    
    model_id = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"
    
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    
    original_model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map={"":0},
        trust_remote_code=True
    )
    
    original_tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
    original_tokenizer.pad_token = original_tokenizer.eos_token
    
    print("원본 모델 로드 완료")
    return original_model, original_tokenizer

In [14]:
# 두 모델의 성능을 비교하는 함수
def compare_models(original_model, original_tokenizer, peft_model, peft_tokenizer, test_texts):
    print("\n모델 성능 비교 시작...")
    print("=" * 60)
    
    results = {
        'original': {'predictions': [], 'times': []},
        'peft': {'predictions': [], 'times': []}
    }
    
    for i, text in enumerate(test_texts):
        print(f"\n테스트 {i+1}: {text}")
        
        # 원본 모델 테스트
        print("   원본 모델 추론 중...")
        start_time = time.time()
        original_pred = evaluate_sentiment(text, original_model, original_tokenizer)
        original_time = time.time() - start_time
        
        # PEFT 모델 테스트
        print("   PEFT 모델 추론 중...")
        start_time = time.time()
        peft_pred = evaluate_sentiment(text, peft_model, peft_tokenizer)
        peft_time = time.time() - start_time
        
        # 결과 저장
        results['original']['predictions'].append(original_pred)
        results['original']['times'].append(original_time)
        results['peft']['predictions'].append(peft_pred)
        results['peft']['times'].append(peft_time)
        
        print(f"   원본: {original_pred} ({original_time:.3f}초)")
        print(f"   PEFT: {peft_pred} ({peft_time:.3f}초)")
        print(f"   일치: {'✅' if original_pred == peft_pred else '❌'}")
    
    return results

In [15]:
# 결과를 분석하고 통계를 출력하는 함수
def analyze_results(results):
    print("\n성능 분석 결과")
    print("=" * 60)
    
    # 예측 일치도 계산
    matches = sum(1 for orig, peft in zip(results['original']['predictions'], results['peft']['predictions']) if orig == peft)
    total = len(results['original']['predictions'])
    agreement = matches / total * 100
    
    # 평균 추론 시간 계산
    avg_original_time = sum(results['original']['times']) / len(results['original']['times'])
    avg_peft_time = sum(results['peft']['times']) / len(results['peft']['times'])
    
    # 예측 분포 계산
    original_pos = sum(1 for p in results['original']['predictions'] if p == '긍정')
    original_neg = sum(1 for p in results['original']['predictions'] if p == '부정')
    peft_pos = sum(1 for p in results['peft']['predictions'] if p == '긍정')
    peft_neg = sum(1 for p in results['peft']['predictions'] if p == '부정')
    
    print(f"예측 일치도: {agreement:.1f}% ({matches}/{total})")
    print(f"평균 추론 시간:")
    print(f"   원본 모델: {avg_original_time:.3f}초")
    print(f"   PEFT 모델: {avg_peft_time:.3f}초")
    print(f"   속도 향상: {avg_original_time/avg_peft_time:.2f}x")
    
    print(f"\n예측 분포:")
    print(f"   원본 모델 - 긍정: {original_pos}개, 부정: {original_neg}개")
    print(f"   PEFT 모델   - 긍정: {peft_pos}개, 부정: {peft_neg}개")
    
    # 메모리 사용량 비교 (대략적)
    print(f"\n메모리 효율성:")
    print(f"   PEFT는 원본 모델 대비 훨씬 적은 메모리를 사용합니다.")
    print(f"   (PEFT 어댑터만 저장하므로 전체 모델 크기의 0.2% 수준)")

### 모델 비교 실행

In [16]:
if 'loaded_model' in locals() and 'loaded_tokenizer' in locals():
    print("=== 모델 성능 비교: 원본 vs PEFT ===\n")
    
    # 비교용 테스트 데이터
    comparison_texts = [
        "이 영화는 정말 재미있었어요!",
        "음식이 맛없고 서비스도 별로였습니다.",
        "제품 품질이 좋고 배송도 빨랐어요.",
        "가격이 너무 비싸고 만족스럽지 않습니다.",
        "배우들의 연기가 훌륭했습니다.",
        "스토리가 너무 지루하고 예측 가능했습니다."
    ]
    
    try:
        # 원본 모델 로드
        original_model, original_tokenizer = load_original_model()
        
        # 모델 비교 실행
        comparison_results = compare_models(
            original_model, original_tokenizer,
            loaded_model, loaded_tokenizer,
            comparison_texts
        )
        
        # 결과 분석
        analyze_results(comparison_results)
        
        print("\n모델 비교 완료!")
        print("PEFT 모델은 원본 모델과 유사한 성능을 유지하면서도")
        print("훨씬 적은 메모리와 저장 공간을 사용합니다.")
        
    except Exception as e:
        print(f"모델 비교 중 오류 발생: {e}")
        
else:
    print("로드된 PEFT 모델이 없습니다. 먼저 모델을 로드해주세요.")


=== 모델 성능 비교: 원본 vs PEFT ===

원본 모델 로드 중...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

원본 모델 로드 완료

모델 성능 비교 시작...

테스트 1: 이 영화는 정말 재미있었어요!
   원본 모델 추론 중...
   PEFT 모델 추론 중...
   원본: 긍정 (1.096초)
   PEFT: 긍정 (0.156초)
   일치: ✅

테스트 2: 음식이 맛없고 서비스도 별로였습니다.
   원본 모델 추론 중...
   PEFT 모델 추론 중...
   원본: 부정 (0.180초)
   PEFT: 부정 (0.156초)
   일치: ✅

테스트 3: 제품 품질이 좋고 배송도 빨랐어요.
   원본 모델 추론 중...
   PEFT 모델 추론 중...
   원본: 긍정 (0.234초)
   PEFT: 긍정 (0.155초)
   일치: ✅

테스트 4: 가격이 너무 비싸고 만족스럽지 않습니다.
   원본 모델 추론 중...
   PEFT 모델 추론 중...
   원본: 부정 (0.179초)
   PEFT: 부정 (0.157초)
   일치: ✅

테스트 5: 배우들의 연기가 훌륭했습니다.
   원본 모델 추론 중...
   PEFT 모델 추론 중...
   원본: 중립 (0.285초)
   PEFT: 긍정 (0.156초)
   일치: ❌

테스트 6: 스토리가 너무 지루하고 예측 가능했습니다.
   원본 모델 추론 중...
   PEFT 모델 추론 중...
   원본: 중립 (0.179초)
   PEFT: 부정 (0.156초)
   일치: ❌

성능 분석 결과
예측 일치도: 66.7% (4/6)
평균 추론 시간:
   원본 모델: 0.359초
   PEFT 모델: 0.156초
   속도 향상: 2.30x

예측 분포:
   원본 모델 - 긍정: 2개, 부정: 2개
   PEFT 모델   - 긍정: 3개, 부정: 3개

메모리 효율성:
   PEFT는 원본 모델 대비 훨씬 적은 메모리를 사용합니다.
   (PEFT 어댑터만 저장하므로 전체 모델 크기의 0.2% 수준)

모델 비교 완료!
PEFT 모델은 원본 모델과 유사한 성능을 유지하면서도
훨씬 적은 메모리

## 11. 추론 최적화
- 메모리 효율적인 추론 방법들

### 함수 정의 

In [17]:
import gc
import psutil
import torch
from contextlib import contextmanager

# 현재 메모리 사용량을 반환하는 함수
def get_memory_usage():
    
    process = psutil.Process()
    memory_info = process.memory_info()
    return memory_info.rss / 1024 / 1024  # MB 단위

In [18]:
# 메모리 효율적인 추론을 위한 컨텍스트 매니저
@contextmanager
def memory_efficient_inference(model, tokenizer):
    
    print("메모리 효율적인 추론 모드 활성화...")
    
    # 추론 전 메모리 정리
    torch.cuda.empty_cache() if torch.cuda.is_available() else None
    gc.collect()
    
    initial_memory = get_memory_usage()
    print(f"   초기 메모리 사용량: {initial_memory:.1f} MB")
    
    try:
        yield model, tokenizer
    finally:
        # 추론 후 메모리 정리
        torch.cuda.empty_cache() if torch.cuda.is_available() else None
        gc.collect()
        
        final_memory = get_memory_usage()
        print(f"   최종 메모리 사용량: {final_memory:.1f} MB")
        print(f"   메모리 변화: {final_memory - initial_memory:+.1f} MB")

In [19]:
# 최적화된 텍스트 생성 함수
def optimized_generate_text(model, tokenizer, prompt, max_length=50, temperature=0.3):
    
    # 입력 토크나이징 (최소한의 토큰만 사용)
    inputs = tokenizer(
        prompt, 
        return_tensors="pt", 
        truncation=True, 
        max_length=256  # 입력 길이 제한
    ).to(model.device)
    
    # 메모리 효율적인 생성 설정
    generation_config = {
        "max_length": inputs['input_ids'].shape[1] + max_length,
        "temperature": temperature,
        "top_p": 0.9,
        "do_sample": True,
        "pad_token_id": tokenizer.eos_token_id,
        "eos_token_id": tokenizer.eos_token_id,
        "repetition_penalty": 1.1,
        "no_repeat_ngram_size": 2,  # 반복 방지
        "early_stopping": True,     # 조기 종료
    }
    
    with torch.no_grad():
        # 메모리 사용량 모니터링
        start_memory = get_memory_usage()
        
        outputs = model.generate(**inputs, **generation_config)
        
        end_memory = get_memory_usage()
        print(f"   생성 중 메모리 사용량: {end_memory - start_memory:+.1f} MB")
    
    # 결과 디코딩
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_text = generated_text[len(prompt):].strip()
    
    return generated_text

In [20]:
# 스트리밍 방식으로 대량의 텍스트를 처리하는 함수
def stream_inference(model, tokenizer, texts, chunk_size=5):
    
    print(f"스트리밍 추론 시작 (청크 크기: {chunk_size})")
    
    results = []
    total_chunks = (len(texts) + chunk_size - 1) // chunk_size
    
    for i in range(0, len(texts), chunk_size):
        chunk_texts = texts[i:i + chunk_size]
        chunk_num = i // chunk_size + 1
        
        print(f"\n청크 {chunk_num}/{total_chunks} 처리 중...")
        
        # 각 청크를 개별적으로 처리하여 메모리 사용량 최소화
        with memory_efficient_inference(model, tokenizer):
            for text in chunk_texts:
                prompt = f"리뷰: {text}\n판별:"
                result = optimized_generate_text(model, tokenizer, prompt)
                
                # 간단한 감정 분석
                sentiment = "긍정" if any(word in result.lower() for word in ["긍정", "좋", "만족"]) else "부정"
                
                results.append({
                    'text': text,
                    'result': result,
                    'sentiment': sentiment
                })
        
        # 청크 간 메모리 정리
        torch.cuda.empty_cache() if torch.cuda.is_available() else None
        gc.collect()
    
    return results

In [21]:
# 다양한 추론 방법의 성능을 벤치마크하는 함수
def benchmark_inference_methods(model, tokenizer, test_texts):
    
    print("⚡ 추론 방법별 성능 벤치마크")
    print("=" * 50)
    
    methods = {
        "일반 추론": lambda: [evaluate_sentiment(text, model, tokenizer) for text in test_texts],
        "최적화된 추론": lambda: [optimized_generate_text(model, tokenizer, f"리뷰: {text}\n판별:") for text in test_texts],
        "스트리밍 추론": lambda: stream_inference(model, tokenizer, test_texts)
    }
    
    results = {}
    
    for method_name, method_func in methods.items():
        print(f"\n{method_name} 테스트 중...")
        
        start_time = time.time()
        start_memory = get_memory_usage()
        
        try:
            if method_name == "스트리밍 추론":
                method_results = method_func()
                predictions = [r['sentiment'] for r in method_results]
            else:
                predictions = method_func()
            
            end_time = time.time()
            end_memory = get_memory_usage()
            
            results[method_name] = {
                'time': end_time - start_time,
                'memory_delta': end_memory - start_memory,
                'predictions': predictions
            }
            
            print(f"   시간: {end_time - start_time:.2f}초")
            print(f"   메모리 변화: {end_memory - start_memory:+.1f} MB")
            print(f"   처리량: {len(test_texts)/(end_time - start_time):.1f} 텍스트/초")
            
        except Exception as e:
            print(f"   ❌ 오류 발생: {e}")
            results[method_name] = {'error': str(e)}
    
    return results

### 추론 최적화 실행

In [22]:
if 'loaded_model' in locals() and 'loaded_tokenizer' in locals():
    print("=== 추론 최적화 예제 ===\n")
    
    # 테스트 데이터
    optimization_test_texts = [
        "이 영화는 정말 재미있었어요!",
        "음식이 맛없고 서비스도 별로였습니다.",
        "제품 품질이 좋고 배송도 빨랐어요.",
        "가격이 너무 비싸고 만족스럽지 않습니다.",
        "배우들의 연기가 훌륭했습니다.",
        "스토리가 너무 지루하고 예측 가능했습니다.",
        "음식점 분위기가 좋고 직원들이 친절했습니다.",
        "배송이 늦고 포장도 엉망이었습니다.",
        "상품이 예상보다 훨씬 좋았습니다.",
        "고객 서비스가 형편없었습니다."
    ]
    
    # 성능 벤치마크 실행
    benchmark_results = benchmark_inference_methods(loaded_model, loaded_tokenizer, optimization_test_texts)
    
    # 결과 요약
    print("\n벤치마크 결과 요약")
    print("=" * 50)
    
    for method, result in benchmark_results.items():
        if 'error' not in result:
            print(f"\n{method}:")
            print(f"   시간: {result['time']:.2f}초")
            print(f"   메모리 변화: {result['memory_delta']:+.1f} MB")
            print(f"   처리량: {len(optimization_test_texts)/result['time']:.1f} 텍스트/초")
        else:
            print(f"\n{method}: ❌ {result['error']}")
    
    print("\n최적화 팁:")
    print("   • 스트리밍 추론: 대량 데이터 처리 시 메모리 효율적")
    print("   • 최적화된 생성: 짧은 응답에 적합")
    print("   • 배치 처리: 중간 규모 데이터에 적합")
    print("   • 메모리 정리: torch.cuda.empty_cache()와 gc.collect() 활용")
    
else:
    print("로드된 모델이 없습니다. 먼저 모델을 로드해주세요.")


=== 추론 최적화 예제 ===

⚡ 추론 방법별 성능 벤치마크

일반 추론 테스트 중...


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


   시간: 1.55초
   메모리 변화: +0.0 MB
   처리량: 6.4 텍스트/초

최적화된 추론 테스트 중...
   생성 중 메모리 사용량: +0.0 MB
   생성 중 메모리 사용량: +0.0 MB
   생성 중 메모리 사용량: +0.0 MB
   생성 중 메모리 사용량: +0.0 MB
   생성 중 메모리 사용량: +0.0 MB
   생성 중 메모리 사용량: +0.0 MB
   생성 중 메모리 사용량: +0.0 MB
   생성 중 메모리 사용량: +0.0 MB
   생성 중 메모리 사용량: +0.0 MB
   생성 중 메모리 사용량: +0.0 MB
   시간: 1.57초
   메모리 변화: +0.0 MB
   처리량: 6.4 텍스트/초

스트리밍 추론 테스트 중...
스트리밍 추론 시작 (청크 크기: 5)

청크 1/2 처리 중...
메모리 효율적인 추론 모드 활성화...
   초기 메모리 사용량: 3289.6 MB
   생성 중 메모리 사용량: +0.0 MB
   생성 중 메모리 사용량: +0.0 MB
   생성 중 메모리 사용량: +0.0 MB
   생성 중 메모리 사용량: +0.0 MB
   생성 중 메모리 사용량: +0.0 MB
   최종 메모리 사용량: 3289.6 MB
   메모리 변화: +0.0 MB

청크 2/2 처리 중...
메모리 효율적인 추론 모드 활성화...
   초기 메모리 사용량: 3289.6 MB
   생성 중 메모리 사용량: +0.0 MB
   생성 중 메모리 사용량: +0.0 MB
   생성 중 메모리 사용량: +0.0 MB
   생성 중 메모리 사용량: +0.0 MB
   생성 중 메모리 사용량: +0.0 MB
   최종 메모리 사용량: 3289.6 MB
   메모리 변화: +0.0 MB
   시간: 2.60초
   메모리 변화: +0.0 MB
   처리량: 3.8 텍스트/초

벤치마크 결과 요약

일반 추론:
   시간: 1.55초
   메모리 변화: +0.0 MB
   처리량: 6.4 텍스트/초

최적화된 추론